## 8- **EMT**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
clean_data= '/content/drive/MyDrive/Colab Notebooks/clean_dataset_oversample'
with open(clean_data, "r") as file:
    lines = file.readlines()
    
clean_full = [line.split() for line in lines]

In [3]:
import random
random.seed(3)
#on travaille avec un échantillon de 100000 tweets
clean_sample = random.sample(clean_full, 100000)
print(len(clean_sample))

100000


In [7]:
cleaned_tweet= list(clean_sample)

for i in range(len(cleaned_tweet)):
    cleaned_tweet[i] = ' '.join(cleaned_tweet[i])

**ETM's original article:**

@article {dieng2019topic,

    title = {Topic modeling in embedding spaces},

    author = {Dieng, Adji B and Ruiz, Francisco J R and Blei, David M},

    journal = {arXiv preprint arXiv: 1907.04907},

    year = {2019}
}


In [ ]:
!pip install -U embedded_topic_model

In [8]:
from embedded_topic_model.utils import preprocessing

# Preprocessing the dataset
vocabulary, train_dataset, _, = preprocessing.create_etm_datasets(
    cleaned_tweet, 
    min_df=0.01, 
    max_df=0.75, 
    train_size=0.85, 
)

In [9]:
#cela a pris 6min35 sur mon PC
%%time
from embedded_topic_model.utils import embedding

# Training word2vec embeddings
embeddings_mapping = embedding.create_word2vec_embedding_from_dataset(cleaned_tweet)

a- **Méthode 1 : avec les embeddings entrainés sur nos données**

In [12]:
%%time
from embedded_topic_model.models.etm import ETM
# Training an ETM instance
etm_instance = ETM(
    vocabulary,
    embeddings=embeddings_mapping, # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=8,
    epochs=15,
    debug_mode=True,
    train_embeddings=False, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)

etm_instance.fit(train_dataset)

Topics before training: [['vouloir', 'femme', 'mal', 'venir', 'pouvoir', 'grand', 'croire', 'jour', 'vie', 'gros'], ['temps', 'bien', 'mettre', 'faire', 'jour', 'passer', 'monde', 'france', 'grand', 'aime'], ['sale', 'monde', 'taire', 'français', 'croire', 'gros', 'passer', 'jour', 'grand', 'aller'], ['aime', 'passer', 'croire', 'taire', 'monde', 'vouloir', 'grand', 'petit', 'fils', 'temps'], ['croire', 'gros', 'vouloir', 'savoir', 'venir', 'devoir', 'bien', 'jour', 'grand', 'an'], ['pute', 'taire', 'france', 'savoir', 'femme', 'chose', 'vouloir', 'fils', 'mettre', 'pouvoir'], ['temps', 'voir', 'passer', 'monde', 'mettre', 'sale', 'femme', 'aime', 'fils', 'pute'], ['temps', 'pute', 'gros', 'petit', 'venir', 'femme', 'jour', 'vrai', 'mal', 'aime']]
Epoch 1 - Learning Rate: 0.005 - KL theta: 0.07 - Rec loss: 5.86 - NELBO: 5.93
Epoch 2 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.82 - NELBO: 5.82
Epoch 3 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.82 - NELBO: 5.82
Epoch 4

In [13]:
%%time
topics = etm_instance.get_topics(20)
topic_coherence = etm_instance.get_topic_coherence()
topic_diversity = etm_instance.get_topic_diversity()

In [14]:
for i in range(8): print(topics[i])

['sale', 'pute', 'gros', 'fils', 'aller', 'faire', 'bien', 'petit', 'voir', 'mettre', 'vouloir', 'aime', 'grand', 'taire', 'pouvoir', 'vie', 'vrai', 'femme', 'venir', 'savoir']
['faire', 'aller', 'bien', 'pouvoir', 'prendre', 'monde', 'savoir', 'taire', 'vouloir', 'falloir', 'devoir', 'voir', 'petit', 'mal', 'femme', 'jour', 'grand', 'mettre', 'passer', 'chose']
['faire', 'aller', 'pouvoir', 'voir', 'venir', 'devoir', 'bien', 'vouloir', 'falloir', 'savoir', 'monde', 'mettre', 'jour', 'chose', 'vrai', 'croire', 'prendre', 'passer', 'taire', 'grand']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'savoir', 'vouloir', 'falloir', 'devoir', 'temps', 'passer', 'monde', 'venir', 'jour', 'mettre', 'vrai', 'prendre', 'femme', 'mal', 'croire']
['français', 'france', 'an', 'grand', 'faire', 'jour', 'pouvoir', 'vouloir', 'devoir', 'aller', 'bien', 'vie', 'mettre', 'passer', 'savoir', 'falloir', 'femme', 'monde', 'petit', 'venir']
['faire', 'aller', 'bien', 'voir', 'pouvoir', 'vouloir', 'falloir', '

In [15]:
print(topic_coherence)

-0.03221466331640218


In [16]:
print(topic_diversity)

0.165


b- **Méthode 2 : avec les embeddings pré-entrainés**

In [17]:
# Training an ETM instance
etm_instance = ETM(
    vocabulary,
    embeddings= '/content/drive/MyDrive/Colab Notebooks/frwiki_20180420_100d.txt', # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=8,
    epochs=20,
    debug_mode=True,
    train_embeddings=True, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)

etm_instance.fit(train_dataset)

Topics before training: [['aller', 'femme', 'jour', 'voir', 'gros', 'vrai', 'sale', 'bien', 'fils', 'grand'], ['jour', 'sale', 'temps', 'bien', 'chose', 'grand', 'pouvoir', 'pute', 'vrai', 'voir'], ['monde', 'devoir', 'vie', 'français', 'prendre', 'mettre', 'vouloir', 'grand', 'taire', 'aime'], ['croire', 'pouvoir', 'petit', 'jour', 'aller', 'voir', 'gros', 'chose', 'faire', 'mal'], ['falloir', 'savoir', 'gros', 'france', 'aller', 'grand', 'taire', 'prendre', 'passer', 'devoir'], ['passer', 'mettre', 'grand', 'vrai', 'savoir', 'devoir', 'petit', 'faire', 'sale', 'jour'], ['voir', 'devoir', 'pute', 'croire', 'femme', 'gros', 'petit', 'passer', 'france', 'chose'], ['france', 'passer', 'gros', 'mettre', 'devoir', 'faire', 'venir', 'petit', 'temps', 'jour']]
Epoch 1 - Learning Rate: 0.005 - KL theta: 0.07 - Rec loss: 5.89 - NELBO: 5.96
Epoch 2 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.81 - NELBO: 5.81
Epoch 3 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.82 - NELBO: 5.82


Remarque , je n'ai pas pu entrainer ce modèle sans mettre l'argument train_embedding (argument optionnel) à True , ce qui semble contredire ce qui est mentionné dans leur commentaire , à moins que j'ai mal compris ??

voici l'erreur que j'ai eue :  unpickling stack underflow 

In [24]:
topics = etm_instance.get_topics(20)
topic_coherence = etm_instance.get_topic_coherence()
topic_diversity = etm_instance.get_topic_diversity()

In [19]:
for i in range(8): print(topics[i])

['faire', 'aller', 'pouvoir', 'bien', 'voir', 'falloir', 'femme', 'savoir', 'vouloir', 'devoir', 'jour', 'france', 'monde', 'mettre', 'sale', 'an', 'grand', 'prendre', 'passer', 'vie']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'falloir', 'savoir', 'femme', 'vouloir', 'devoir', 'france', 'jour', 'mettre', 'sale', 'an', 'prendre', 'grand', 'vie', 'français', 'monde']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'savoir', 'falloir', 'femme', 'vouloir', 'devoir', 'an', 'jour', 'france', 'sale', 'mettre', 'prendre', 'français', 'grand', 'vie', 'monde']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'falloir', 'femme', 'jour', 'devoir', 'france', 'savoir', 'vouloir', 'mettre', 'temps', 'an', 'prendre', 'français', 'monde', 'grand', 'passer']
['faire', 'aller', 'pouvoir', 'bien', 'voir', 'femme', 'falloir', 'savoir', 'vouloir', 'jour', 'devoir', 'france', 'an', 'mettre', 'prendre', 'temps', 'monde', 'français', 'grand', 'vie']
['faire', 'aller', 'pouvoir', 'bien', 'voir', 'femme', 'falloi

In [20]:
print(topic_coherence)

-0.04199612969329938


In [21]:
topic_diversity

0.165

In [23]:
from embedded_topic_model.models.etm import ETM
# Training an ETM instance
etm_instance = ETM(
    vocabulary,
    embeddings=embeddings_mapping, # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=8,
    epochs=20,
    debug_mode=True,
    train_embeddings=True, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)

etm_instance.fit(train_dataset)

Topics before training: [['aller', 'femme', 'jour', 'voir', 'gros', 'vrai', 'sale', 'bien', 'fils', 'grand'], ['jour', 'sale', 'temps', 'bien', 'chose', 'grand', 'pouvoir', 'pute', 'vrai', 'voir'], ['monde', 'devoir', 'vie', 'français', 'prendre', 'mettre', 'vouloir', 'grand', 'taire', 'aime'], ['croire', 'pouvoir', 'petit', 'jour', 'aller', 'voir', 'gros', 'chose', 'faire', 'mal'], ['falloir', 'savoir', 'gros', 'france', 'aller', 'grand', 'taire', 'prendre', 'passer', 'devoir'], ['passer', 'mettre', 'grand', 'vrai', 'savoir', 'devoir', 'petit', 'faire', 'sale', 'jour'], ['voir', 'devoir', 'pute', 'croire', 'femme', 'gros', 'petit', 'passer', 'france', 'chose'], ['france', 'passer', 'gros', 'mettre', 'devoir', 'faire', 'venir', 'petit', 'temps', 'jour']]
Epoch 1 - Learning Rate: 0.005 - KL theta: 0.07 - Rec loss: 5.89 - NELBO: 5.96
Epoch 2 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.81 - NELBO: 5.81
Epoch 3 - Learning Rate: 0.005 - KL theta: 0.0 - Rec loss: 5.82 - NELBO: 5.82


In [25]:
%%time
topics = etm_instance.get_topics(20)
topic_coherence = etm_instance.get_topic_coherence()
topic_diversity = etm_instance.get_topic_diversity()

CPU times: user 1min 33s, sys: 136 ms, total: 1min 33s
Wall time: 1min 34s


In [26]:
for i in range(8): print(topics[i])

['faire', 'aller', 'pouvoir', 'bien', 'voir', 'falloir', 'femme', 'savoir', 'vouloir', 'devoir', 'jour', 'france', 'monde', 'mettre', 'sale', 'an', 'grand', 'prendre', 'passer', 'vie']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'falloir', 'savoir', 'femme', 'vouloir', 'devoir', 'france', 'jour', 'mettre', 'sale', 'an', 'prendre', 'grand', 'vie', 'français', 'monde']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'savoir', 'falloir', 'femme', 'vouloir', 'devoir', 'an', 'jour', 'france', 'sale', 'mettre', 'prendre', 'français', 'grand', 'vie', 'monde']
['faire', 'aller', 'pouvoir', 'voir', 'bien', 'falloir', 'femme', 'jour', 'devoir', 'france', 'savoir', 'vouloir', 'mettre', 'temps', 'an', 'prendre', 'français', 'monde', 'grand', 'passer']
['faire', 'aller', 'pouvoir', 'bien', 'voir', 'femme', 'falloir', 'savoir', 'vouloir', 'jour', 'devoir', 'france', 'an', 'mettre', 'prendre', 'temps', 'monde', 'français', 'grand', 'vie']
['faire', 'aller', 'pouvoir', 'bien', 'voir', 'femme', 'falloi

In [27]:
# Training an ETM instance
etm_instance = ETM(
    vocabulary,
    embeddings= '/content/drive/MyDrive/Colab Notebooks/frwiki_20180420_100d.txt', # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=8,
    epochs=20,
    debug_mode=True,
    train_embeddings=False, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)

etm_instance.fit(train_dataset)

Reading embeddings from word2vec file...


FileNotFoundError: ignored